## Introduction

This tutorial demonstrates how to create an interactive dashboard for visualizing relationships between economic development, population growth, CO2 emissions, and climate change. The dashboard combines multiple visualization techniques to tell a compelling data story through an intuitive, interactive interface.

By the end of this tutorial, you'll learn:
- How to implement and combine different visualization types
- When to use each visualization technique for effective data storytelling
- How to create interactive elements using Dash and Plotly
- Best practices for dashboard design and implementation

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
import glob
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import os
# Set the data directory
DATA_DIR = "Data"

## Step 1: Loading and Processing GDP and Population Data

**Purpose:**
Load and prepare economic and population data sourced from the World Bank for further analysis.

**Processing Steps:**

- **Import Data:**
  - Read World Bank CSV data from the `Data` directory.

- **Transform Data:**
  - Convert the data from wide format (years as columns) to long format using `pandas.melt()`.
  - Extract numerical year values and ensure they're integers.
  - Convert non-numeric entries to numeric values, marking invalid entries as `NaN`.

- **Pivot Data:**
  - Pivot the data into a clear structure with individual economic metrics as separate columns.

**Example Code:**
```python
df_long = pd.melt(
    df,
    id_vars=['Country Name', 'Country Code', 'Series Name', 'Series Code'],
    var_name='Year',
    value_name='Value'
)

df_long['Year'] = df_long['Year'].str.extract(r'(\d{4})').astype(int)
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')

df_pivot = df_long.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Series Name',
    values='Value'
).reset_index()

In [2]:
# Load GDP and Population data
def load_worldbank_data():
    # Look for World Bank data files in the Data directory
    wb_files = glob.glob(os.path.join(DATA_DIR, "9e282781-6439-45eb-9429-239b07b477e9_Data.csv"))

    if not wb_files:
        print("No World Bank data files found in Data directory.")
        return None

    # Use the first file found
    file_path = wb_files[0]
    print(f"Loading World Bank data from: {file_path}")

    # Read the file based on extension
    if file_path.endswith('.xlsx'):
        df = pd.read_excel(file_path)
    else:
        df = pd.read_csv(file_path)

    # Melt the data from wide to long format
    id_vars = ['Country Name', 'Country Code', 'Series Name', 'Series Code']
    year_cols = [col for col in df.columns if str(col).startswith('19') or str(col).startswith('20')]

    print(f"Processing years: {year_cols}")

    df_long = pd.melt(
        df,
        id_vars=id_vars,
        value_vars=year_cols,
        var_name='Year',
        value_name='Value'
    )

    # Clean up year column - extract numeric year
    df_long['Year'] = df_long['Year'].str.extract(r'(\d{4})').astype(int) if df_long['Year'].dtype == 'object' else df_long['Year']

    # Convert Value column to numeric, replacing non-numeric values with NaN
    df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')

    # Pivot to have series as columns
    df_pivot = df_long.pivot_table(
        index=['Country Name', 'Country Code', 'Year'],
        columns='Series Name',
        values='Value',
        aggfunc='first'  # Use 'first' instead of default 'mean'
    ).reset_index()

    # Rename columns for clarity
    df_pivot.columns.name = None

    return df_pivot

## Step 2: Loading and Processing CO₂ Emissions Data
**Purpose:**
Integrate CO₂ emissions data from global databases such as Our World in Data or the Global Carbon Project.

**Processing Steps:**

- **Import Data:**
   - Load CO₂ emissions data CSV from the Data directory.

- **Data Cleaning:**

  - Identify essential columns (Country Name, Country Code, Year, CO₂ Emissions).

  - Standardize column names for consistency.

  - Adapt to variations in dataset formats from different sources.

**Example Code:**

```python
df = pd.read_csv(file_path)

# Standardize columns
df = df.rename(columns={
    'Entity': 'Country Name',
    'Code': 'Country Code',
    'Year': 'Year',
    co2_column: 'CO2 Emissions (kt)'
})

df = df[['Country Name', 'Country Code', 'Year', 'CO2 Emissions (kt)']]

In [3]:
# Function to load CO2 emissions data
def load_co2_data():
    # Look for CO2 data files in the Data directory
    co2_files = glob.glob(os.path.join(DATA_DIR, "*co2*.csv")) or \
                glob.glob(os.path.join(DATA_DIR, "*carbon*.csv")) or \
                glob.glob(os.path.join(DATA_DIR, "*emission*.csv"))

    if not co2_files:
        print("No CO2 data files found in Data directory.")
        return None

    # Use the first file found
    file_path = co2_files[0]
    print(f"Loading CO2 data from: {file_path}")

    # Read the data
    df = pd.read_csv(file_path)

    # Different CO2 data sources have different formats
    # Check for Our World in Data format
    if 'Entity' in df.columns and 'Code' in df.columns:
        # Our World in Data format
        co2_column = next((col for col in df.columns if "co2" in col.lower() or "emission" in col.lower()), None)

        if co2_column:
            df = df.rename(columns={
                'Entity': 'Country Name',
                'Code': 'Country Code',
                'Year': 'Year',
                co2_column: 'CO2 Emissions (kt)'
            })
            df = df[['Country Name', 'Country Code', 'Year', 'CO2 Emissions (kt)']]

    # Check for Global Carbon Project format
    elif 'country' in df.columns and 'year' in df.columns:
        emission_col = next((col for col in df.columns if "co2" in col.lower() or "emission" in col.lower()), None)

        if emission_col:
            df = df.rename(columns={
                'country': 'Country Name',
                'iso_code': 'Country Code' if 'iso_code' in df.columns else None,
                'year': 'Year',
                emission_col: 'CO2 Emissions (kt)'
            })
            cols = ['Country Name', 'Year', 'CO2 Emissions (kt)']
            if 'Country Code' in df.columns:
                cols.insert(1, 'Country Code')
            df = df[cols]

    return df

In [4]:
def load_temperature_data():
    # Look for temperature data files in the Data directory
    temp_files = glob.glob(os.path.join(DATA_DIR, "*temp*.csv")) or \
                 glob.glob(os.path.join(DATA_DIR, "*climate*.csv")) or \
                 glob.glob(os.path.join(DATA_DIR, "*berkeley*.csv"))

    if not temp_files:
        print("No temperature data files found in Data directory.")
        return None

    # Use the first file found
    file_path = temp_files[0]
    print(f"Loading temperature data from: {file_path}")

    # Read the data
    df = pd.read_csv(file_path)

    # Print column names to help diagnose the issue
    print(f"Temperature data columns: {df.columns.tolist()}")

    # Check for date/time column variations
    date_cols = [col for col in df.columns if any(date_term in col.lower()
                                                  for date_term in ['year', 'date', 'time', 'dt'])]

    if date_cols:
        print(f"Found potential date columns: {date_cols}")
        date_col = date_cols[0]  # Use the first date-like column

        # Check if it's a datetime column and convert to datetime if needed
        if pd.api.types.is_object_dtype(df[date_col]):
            try:
                df['Year'] = pd.to_datetime(df[date_col]).dt.year
                print(f"Extracted years from {date_col} column")
            except Exception as e:
                print(f"Error extracting year from {date_col}: {e}")
                return None
        else:
            # Just rename the column
            df = df.rename(columns={date_col: 'Year'})

        # Look for temperature column
        temp_cols = [col for col in df.columns if any(temp_term in col.lower()
                                                      for temp_term in ['temp', 'anomaly', 'land', 'ocean'])]

        if temp_cols:
            # Prefer LandAndOceanAverageTemperature if available
            if 'LandAndOceanAverageTemperature' in temp_cols:
                temp_col = 'LandAndOceanAverageTemperature'
            else:
                temp_col = temp_cols[0]

            print(f"Using temperature column: {temp_col}")

            # Create a new dataframe with just the columns we need
            new_df = df[['Year', temp_col]].copy()
            new_df = new_df.rename(columns={temp_col: 'Temperature Anomaly'})

            # Drop rows with missing temperature data
            new_df = new_df.dropna()

            # Group by year to get annual averages if data is monthly
            if len(new_df) > 500:  # Likely monthly data
                print("Converting monthly temperature data to annual averages")
                new_df = new_df.groupby('Year')['Temperature Anomaly'].mean().reset_index()

            return new_df

    print("Warning: Could not identify year and temperature columns in the data.")
    return None

In [5]:
def create_merged_dataset(gdp_pop_df=None, co2_df=None, temp_df=None):
    # Start with GDP/Population data if available
    if gdp_pop_df is not None:
        merged_df = gdp_pop_df.copy()

        # Merge with CO2 data if available
        if co2_df is not None:
            # Ensure consistent 'Year' type for merging
            merged_df['Year'] = merged_df['Year'].astype(int)
            co2_df['Year'] = co2_df['Year'].astype(int)

            # Check if 'Country Code' is available for merging
            if 'Country Code' in co2_df.columns:
                merged_df = pd.merge(
                    merged_df,
                    co2_df,
                    on=['Country Name', 'Country Code', 'Year'],
                    how='outer'
                )
            else:
                # Merge on Country Name and Year only
                merged_df = pd.merge(
                    merged_df,
                    co2_df,
                    on=['Country Name', 'Year'],
                    how='outer'
                )
    else:
        # Start with CO2 data if GDP/Population not available
        if co2_df is not None:
            merged_df = co2_df.copy()
        else:
            print("No GDP/Population or CO2 data available for merging.")
            return None

    # Merge with temperature data if available
    if temp_df is not None and 'Year' in temp_df.columns:
        # Ensure consistent 'Year' type for merging
        merged_df['Year'] = merged_df['Year'].astype(int)
        # No need to convert temp_df['Year'] since it's already an int from the load_temperature_data function

        # Temperature data is typically global, so merge on Year only
        merged_df = pd.merge(
            merged_df,
            temp_df,
            on='Year',
            how='left'
        )

    # Calculate derived metrics
    if all(col in merged_df.columns for col in ['GDP (Current US$)', 'Population, total']):
        # GDP per capita
        merged_df['GDP per capita'] = merged_df['GDP (Current US$)'] / merged_df['Population, total']

    if all(col in merged_df.columns for col in ['CO2 Emissions (kt)', 'Population, total']):
        # CO2 per capita (tonnes per person)
        merged_df['CO2 per capita'] = merged_df['CO2 Emissions (kt)'] / merged_df['Population, total'] * 1000

    if all(col in merged_df.columns for col in ['CO2 Emissions (kt)', 'GDP (Current US$)']):
        # CO2 intensity (kg per $ of GDP)
        merged_df['CO2 intensity'] = merged_df['CO2 Emissions (kt)'] * 1000000 / merged_df['GDP (Current US$)']

    return merged_df

In [6]:
gdp_pop_df = load_worldbank_data()
co2_df = load_co2_data()
temp_df = load_temperature_data()
df = create_merged_dataset(gdp_pop_df, co2_df, temp_df)

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define available metrics for visualization
metrics = {
    'GDP (Current US$)': 'GDP (Current US$)',
    'Population, total': 'Population',
    'CO2 Emissions (kt)': 'CO2 Emissions',
    'GDP per capita': 'GDP per Capita',
    'CO2 per capita': 'CO2 per Capita',
    'CO2 intensity': 'CO2 Intensity',
    'Temperature Anomaly': 'Temperature Anomaly'
}

# Get list of countries and years
countries = sorted(df['Country Name'].unique())
years = sorted(df['Year'].unique())

Loading World Bank data from: Data\9e282781-6439-45eb-9429-239b07b477e9_Data.csv
Processing years: ['1960 [YR1960]', '1961 [YR1961]', '1962 [YR1962]', '1963 [YR1963]', '1964 [YR1964]', '1965 [YR1965]', '1966 [YR1966]', '1967 [YR1967]', '1968 [YR1968]', '1969 [YR1969]', '1970 [YR1970]', '1971 [YR1971]', '1972 [YR1972]', '1973 [YR1973]', '1974 [YR1974]', '1975 [YR1975]', '1976 [YR1976]', '1977 [YR1977]', '1978 [YR1978]', '1979 [YR1979]', '1980 [YR1980]', '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]', '1984 [YR1984]', '1985 [YR1985]', '1986 [YR1986]', '1987 [YR1987]', '1988 [YR1988]', '1989 [YR1989]', '1990 [YR1990]', '1991 [YR1991]', '1992 [YR1992]', '1993 [YR1993]', '1994 [YR1994]', '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]', '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]', '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]', '2011 [YR2011]', '2012 [YR2012]',

In [7]:
df.head()

,Country Name,Country Code,Year,GDP (current US$),"Population, total",CO2 Emissions (kt),CO2 per capita
0,Afghanistan,AFG,1949,NaN,NaN,14656.0,NaN
1,Afghanistan,AFG,1950,NaN,NaN,84272.0,NaN
2,Afghanistan,AFG,1951,NaN,NaN,91600.0,NaN
3,Afghanistan,AFG,1952,NaN,NaN,91600.0,NaN
4,Afghanistan,AFG,1953,NaN,NaN,106256.0,NaN


### Multiple Chart Types Working Together

Our dashboard leverages four complementary visualization types to provide multiple perspectives on the climate and economic development data:

#### 1. Choropleth Map
The choropleth map provides a geographic view of our data, color-coding countries based on the selected metric. This visualization gives users an immediate global perspective on the distribution of values across different regions.

**Best Use Cases:**
- Showing geographic patterns and regional variations
- Identifying outlier countries or regions
- Providing context for country-specific analyses

In [8]:
# Cell 2: Choropleth Map Function
def create_choropleth_map(df, selected_year, selected_metric, metrics):
    """Create a choropleth map for the selected year and metric."""
    # Filter data for the selected year
    filtered_df = df[df['Year'] == selected_year].dropna(subset=[selected_metric])

    # Create choropleth map
    fig = px.choropleth(
        filtered_df,
        locations='Country Code',
        color=selected_metric,
        hover_name='Country Name',
        color_continuous_scale=px.colors.sequential.Plasma,
        title=f'{metrics[selected_metric]} by Country ({selected_year})'
    )

    fig.update_layout(
        coloraxis_colorbar=dict(title=metrics[selected_metric]),
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='equirectangular'
        )
    )

    return fig

# Test the function with sample data - using yearly totals
sample_year = 2015  # Use an available year in your data
sample_metric = 'CO2 Emissions (kt)'  # Use total emissions instead of per capita

choropleth_fig = create_choropleth_map(df, sample_year, sample_metric, metrics)
choropleth_fig.show()  # Display the figure in Jupyter

#### 2. Scatter Plot
Our scatter plot reveals relationships between two variables (such as GDP and CO2 emissions), with point size representing a third variable (population). This allows users to identify correlations, outliers, and clusters.

**Best Use Cases:**
- Analyzing relationships between different metrics
- Identifying countries that deviate from general trends
- Visualizing multiple dimensions simultaneously

In [9]:
# Cell 3: Scatter Plot Function - Using yearly totals
def create_scatter_plot(df, selected_year, selected_countries):
    """Create a scatter plot of GDP vs CO2 emissions with population as size."""
    # Filter data for the selected year
    filtered_df = df[df['Year'] == selected_year].dropna(
        subset=['GDP (current US$)', 'CO2 Emissions (kt)', 'Population, total']
    )

    # Highlight selected countries
    filtered_df['Selected'] = filtered_df['Country Name'].isin(selected_countries)

    # Create scatter plot with yearly totals
    fig = px.scatter(
        filtered_df,
        x='GDP (current US$)',
        y='CO2 Emissions (kt)',
        size='Population, total',
        color='Selected',
        hover_name='Country Name',
        log_x=True,  # Log scale for GDP
        log_y=True,  # Log scale for emissions
        size_max=60,
        color_discrete_map={True: 'red', False: 'blue'},
        title=f'GDP vs CO2 Emissions ({selected_year})'
    )

    fig.update_layout(
        xaxis_title='GDP (Current US$, log scale)',
        yaxis_title='CO2 Emissions (kt, log scale)',
        legend_title='Selected Countries'
    )

    return fig

# Test the function with sample data
sample_countries = ['United States', 'China', 'India', 'Germany', 'Brazil']  # Use available countries

scatter_fig = create_scatter_plot(df, sample_year, sample_countries)
scatter_fig.show()  # Display the figure in Jupyter

#### 3. Time Series Line Chart
The time series chart tracks how metrics change over time for selected countries, which is crucial for understanding trends and making comparisons of historical patterns.

**Best Use Cases:**
- Tracking changes over time
- Comparing rates of change across countries
- Identifying key turning points or policy impacts

In [10]:
# Cell 4: Time Series Function
def create_time_series(df, selected_countries, selected_metric, metrics):
    """Create a time series plot for selected countries and metric."""
    # Filter data for selected countries
    filtered_df = df[df['Country Name'].isin(selected_countries)].dropna(subset=[selected_metric])

    # Create time series plot
    fig = px.line(
        filtered_df,
        x='Year',
        y=selected_metric,
        color='Country Name',
        title=f'{metrics[selected_metric]} Over Time'
    )

    fig.update_layout(
        xaxis_title='Year',
        yaxis_title=metrics[selected_metric],
        legend_title='Country'
    )

    return fig

# Test the function with sample data - using yearly totals
time_series_fig = create_time_series(df, sample_countries, 'CO2 Emissions (kt)', metrics)
time_series_fig.show()  # Display the figure in Jupyter

#### 4. Bar Chart
Our bar chart provides direct comparisons between selected countries for a given year and metric, making differences immediately apparent.

**Best Use Cases:**
- Direct comparison between a small number of entities
- Ranking countries by a specific metric
- Highlighting specific differences

In [11]:
# Cell 5: Bar Chart Function
def create_bar_chart(df, selected_year, selected_countries, selected_metric, metrics):
    """Create a bar chart comparing countries for the selected metric and year."""
    # Filter data for selected countries and year
    filtered_df = df[(df['Country Name'].isin(selected_countries)) &
                     (df['Year'] == selected_year)].dropna(subset=[selected_metric])

    # Sort countries by the selected metric
    filtered_df = filtered_df.sort_values(by=selected_metric)

    # Create bar chart
    fig = px.bar(
        filtered_df,
        x='Country Name',
        y=selected_metric,
        title=f'{metrics[selected_metric]} by Country ({selected_year})'
    )

    fig.update_layout(
        xaxis_title='Country',
        yaxis_title=metrics[selected_metric]
    )

    return fig

# Test the function with sample data - using yearly totals
bar_fig = create_bar_chart(df, sample_year, sample_countries, 'CO2 Emissions (kt)', metrics)
bar_fig.show()  # Display the figure in Jupyter

In [12]:
# Cell 6: Advanced Visualization - CO2 Emissions vs GDP Over Time
def create_bubble_animation(df, selected_countries=None):
    """Create an animated bubble chart showing CO2 emissions vs GDP over time."""
    # Filter data
    if selected_countries:
        filtered_df = df[df['Country Name'].isin(selected_countries)].dropna(
            subset=['GDP (current US$)', 'CO2 Emissions (kt)', 'Population, total']
        )
    else:
        # Use top 20 countries by population if none specified
        top_countries = df.groupby('Country Name')['Population, total'].max().nlargest(20).index.tolist()
        filtered_df = df[df['Country Name'].isin(top_countries)].dropna(
            subset=['GDP (Current US$)', 'CO2 Emissions (kt)', 'Population, total']
        )

    # Create animated bubble chart
    fig = px.scatter(
        filtered_df,
        x='GDP (current US$)',
        y='CO2 Emissions (kt)',
        size='Population, total',
        color='Country Name',
        hover_name='Country Name',
        log_x=True,
        log_y=True,
        size_max=60,
        animation_frame='Year',
        title='CO2 Emissions vs GDP Over Time'
    )

    fig.update_layout(
        xaxis_title='GDP (Current US$, log scale)',
        yaxis_title='CO2 Emissions (kt, log scale)'
    )

    return fig

# Test the animation with sample countries
animation_fig = create_bubble_animation(df, sample_countries)
animation_fig.show()  # Display the animation in Jupyter

In [13]:
# Cell 6: Dashboard Callbacks
# These would be integrated in the final Dash app

# Callback for Choropleth Map
@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('year-slider', 'value'),
     Input('map-metric-dropdown', 'value')]
)
def update_map(selected_year, selected_metric):
    return create_choropleth_map(df, selected_year, selected_metric, metrics)

# Callback for Scatter Plot
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value')]
)
def update_scatter(selected_year, selected_countries):
    return create_scatter_plot(df, selected_year, selected_countries)

# Callback for Time Series
@app.callback(
    Output('time-series', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('map-metric-dropdown', 'value')]
)
def update_time_series(selected_countries, selected_metric):
    return create_time_series(df, selected_countries, selected_metric, metrics)

# Callback for Bar Chart
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value'),
     Input('map-metric-dropdown', 'value')]
)
def update_bar_chart(selected_year, selected_countries, selected_metric):
    return create_bar_chart(df, selected_year, selected_countries, selected_metric, metrics)

In [19]:
# Run the Dashboard (for when you're ready to integrate everything)
# Define the complete layout here
# Create and run the dashboard using JupyterDash
# This cell should only be run once
from jupyter_dash import JupyterDash
# Initialize a JupyterDash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H1("Climate Change, Economic Development, and CO2 Emissions Dashboard"),
            html.P("Explore relationships between GDP, population, CO2 emissions, and temperature anomalies")
        ])
    ]),

    # Controls
    dbc.Row([
        dbc.Col([
            html.Label("Select Year:"),
            dcc.Slider(
                id='year-slider',
                min=min(years),
                max=max(years),
                value=sample_year,  # Use our sample year as default
                marks={year: str(year) for year in range(min(years), max(years)+1, 15)},
                step=10
            )
        ])
    ]),

    dbc.Row([
        dbc.Col([
            html.Label("Select Countries:"),
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': country, 'value': country} for country in countries],
                value=sample_countries,  # Default selection
                multi=True
            )
        ], width=6),

        dbc.Col([
            html.Label("Select Metric:"),
            dcc.Dropdown(
                id='map-metric-dropdown',
                options=[{'label': label, 'value': key} for key, label in metrics.items()],
                value=sample_metric  # Default value
            )
        ], width=6)
    ]),

    # Visualizations
    dbc.Row([
        dbc.Col([
            html.H4("Global Distribution"),
            dcc.Graph(id='choropleth-map')
        ], width=12)
    ]),

    dbc.Row([
        dbc.Col([
            html.H4("Relationship Analysis"),
            dcc.Graph(id='scatter-plot')
        ], width=6),

        dbc.Col([
            html.H4("Historical Trends"),
            dcc.Graph(id='time-series')
        ], width=6)
    ]),

    dbc.Row([
        dbc.Col([
            html.H4("Comparison by Country"),
            dcc.Graph(id='bar-chart')
        ], width=12)
    ])
], fluid=True)

# Define callbacks
@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('year-slider', 'value'),
     Input('map-metric-dropdown', 'value')]
)
def update_map(selected_year, selected_metric):
    return create_choropleth_map(df, selected_year, selected_metric, metrics)

@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value')]
)
def update_scatter(selected_year, selected_countries):
    return create_scatter_plot(df, selected_year, selected_countries)

@app.callback(
    Output('time-series', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('map-metric-dropdown', 'value')]
)
def update_time_series(selected_countries, selected_metric):
    return create_time_series(df, selected_countries, selected_metric, metrics)

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('year-slider', 'value'),
     Input('country-dropdown', 'value'),
     Input('map-metric-dropdown', 'value')]
)
def update_bar_chart(selected_year, selected_countries, selected_metric):
    return create_bar_chart(df, selected_year, selected_countries, selected_metric, metrics)

# Run the app in JupyterDash mode
app.run(mode='inline')  # Use 'inline', 'external', or 'jupyterlab' modes

c:\Users\zacha\DataspellProjects\Credit_Card_Fraud_Detection\venv\Lib\site-packages\dash\dash.py:582: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

